In [6]:
import os
from pathlib import Path

from utils import check_search_output, patch_config


from olive.workflows import run as olive_run

olive_config = patch_config("bert_workflow_cpu.json")
footprint = olive_run(olive_config)
check_search_output(footprint)


/home/emmanuel/.conda/envs/emmanuel-onnx/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-06-29 00:10:57,266] [DEBUG] [engine.py:577:resolve_goals] Resolving goals: {'accuracy': {'accuracy': None}, 'latency': {'avg': None}}
[2023-06-29 00:10:57,267] [DEBUG] [engine.py:596:resolve_goals] No baseline got as no goal is provided the the goal is threshold
[2023-06-29 00:10:57,277] [DEBUG] [engine.py:498:run_search] Step 1 with search point {'OnnxConversion': {}, 'OrtTransformersOptimization': {}, 'OnnxQuantization': {'quant_mode': 'static', 'calibrate_method': 'MinMax', 'quant_format': 'QOperator', 'MatMulConstBOnly': False, 'weight_type': 'QUInt8', 'activation_type': 'QUInt8', 'per_channel': False, 'reduce_range': False, 'optimize_model': True, 'quant_preprocess': True}, 'OrtPerfTuning': {}} ...
[2023-06-29 00:10:57,277] [INFO] [engine.py:837:_run_pass] Running pass OnnxConversion
[2023-06-29 00:10:57,278] [DEBUG] [engine.py:703:_prepare_non_local_model] Model path is None, local or string name. No need to prepare
[2023-06-29 00:10:57,279] [DEBUG] [__init__.py:582:get_dum

/home/emmanuel/.conda/envs/emmanuel-onnx/lib/python3.8/site-packages/optuna/samplers/_tpe/sampler.py:278: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
/home/emmanuel/.conda/envs/emmanuel-onnx/lib/python3.8/site-packages/optuna/samplers/_tpe/sampler.py:289: ExperimentalWarning: ``group`` option is an experimental feature. The interface can change in the future.
  warnings.warn(


[2023-06-29 00:10:57,717] [DEBUG] [conversion.py:73:_run_for_config] Using hf config to get io_config for the model.
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

[2023-06-29 00:11:06,068] [INFO] [engine.py:837:_run_pass] Running pass OrtTransformersOptimization
[2023-06-29 00:11:06,070] [DEBUG] [engine.py:703:_prepare_non_local_model] Model path is None, local or string name. No need to prepare
[2023-06-29 00:11:24,090] [INFO] [engine.py:837:_run_pass] Running pass OnnxQuantization
[2023-06-29 00:11:24,094] [DEBUG] [engine.py:703:_prepare_non_local_model] Model path is None, local or string name. No need to prepare
[2023-06-29 00:11:24,095] [INFO] [quantization.py:333:_run_for_config] Preprocessing model for quantization
[2023-06-29 00:12:08,588] [INFO] [engine.py:837:_run_pass] Running pass OrtPerfTuning
[2023-06-29 00:12:08,592] 

In [24]:
footprint
#get first item from dict
fooott = list(footprint.values())[0] 

for node in fooott.nodes.values():
    print(node.metrics)
    for node in node.metrics:
        print(node)
   # print(type(node.metrics.value))


value=MetricResult(__root__={'accuracy-accuracy': SubMetricResult(value=0.8455882352941176, priority=1, higher_is_better=True), 'latency-avg': SubMetricResult(value=24.17589, priority=2, higher_is_better=False)}) cmp_direction={'accuracy-accuracy': 1, 'latency-avg': -1} is_goals_met=True
('value', MetricResult(__root__={'accuracy-accuracy': SubMetricResult(value=0.8455882352941176, priority=1, higher_is_better=True), 'latency-avg': SubMetricResult(value=24.17589, priority=2, higher_is_better=False)}))
('cmp_direction', {'accuracy-accuracy': 1, 'latency-avg': -1})
('is_goals_met', True)


In [26]:
from olive.engine.footprint import Footprint

In [27]:
pf = Footprint.from_file("models/bert_workflow_cpu/cpu-cpu_pareto_frontier_footprints.json")

In [69]:
metrics_of_interest = ['accuracy-accuracy', 'latency-avg']
# gather the metrics from all pareto frontier nodes
all_metrics = []
# we iterate over the nodes in the pareto frontier
for node in pf.nodes.values():
    metrics = []
    # collecting the metrics of interest
    for name in metrics_of_interest:
        # (value of metric * direction of comparison)
        # now higher is better for all metrics
        metrics.append(node.metrics.value[name].value * node.metrics.cmp_direction[name])
    all_metrics.append(metrics)
# sort the metrics
# this sorts it
sorted_metrics = sorted(all_metrics, reverse=True)
# get best metrics
# last one is the best
best_metrics = sorted_metrics[0]
best_metrics


[0.8455882352941176, -24.17589]

In [70]:
import json

#open best metrics json
with open('best_metrics.json') as f:
    data = json.load(f)
    print(data[0], data[1])
    print(best_metrics[0], best_metrics[1])
    if best_metrics[0] > data[0] and best_metrics[1] < data[1]:
        best_metrics = data
#save best metrics to json
with open('best_metrics.json', 'w') as f:
    json.dump(best_metrics, f)
    

0.1 -20
0.8455882352941176 -24.17589
